In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Working with Parallel Function Calls and Multiple Function Responses in Gemini (Asynchronous Example)

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling_async.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ffunction-calling%2Fparallel_function_calling_async.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/function-calling/parallel_function_calling_async.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/parallel_function_calling_async.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Kristopher Overholt](https://github.com/koverholt) |

## Overview

Gemini is a family of generative AI models developed by Google DeepMind designed for multimodal use cases. The Gemini API provides access to the Gemini Pro and Gemini Pro Vision models.

[Function Calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) in Gemini allows you to describe functions in your code and pass those descriptions to the language model. The model's response includes the name of a function that matches the description along with the arguments to call it with.

This asynchronous tutorial demonstrates how to handle parallel function calls within Gemini Function Calling, including:

- Handling parallel function calls for repeated functions asynchronously
- Working with parallel function calls across multiple functions asynchronously
- Extracting multiple function calls from a Gemini response
- Making multiple asynchronous function calls and returning responses to Gemini

### What is parallel function calling?

In previous Gemini Pro models (prior to May 2024), Gemini returned chained function calls if multiple function calls were needed before returning a natural language summary. Chained function calls require sequential processing.

Recent Gemini Pro models (from May 2024 onward) support parallel function calling, allowing multiple function call responses within a single response object. This enables parallelizing API calls or other actions in your application code, reducing the need for sequential processing.

<img src="https://storage.googleapis.com/github-repo/generative-ai/gemini/function-calling/parallel-function-calling-in-gemini.png">

## Getting Started


### Install Vertex AI SDK and Other Required Packages

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform wikipedia aiohttp

### Restart Runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. Run the cell below to restart the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate Your Notebook Environment (Colab Only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [3]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud Project Information and Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Code Examples

### Import Libraries

In [5]:
from typing import Any, Dict
import asyncio
from IPython.display import Markdown, display
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Part,
    Tool,
)
import wikipedia

### Define Helper Function

In [6]:
# Helper function to extract one or more function calls from a Gemini Function Call response
def extract_function_calls(response: GenerationResponse) -> list[Dict[str, Dict[str, Any]]]:
    function_calls: list[Dict[str, Dict[str, Any]]] = []
    if response.candidates[0].function_calls:
        for function_call in response.candidates[0].function_calls:
            function_call_dict: Dict[str, Dict[str, Any]] = {function_call.name: {}}
            for key, value in function_call.args.items():
                function_call_dict[function_call.name][key] = value
            function_calls.append(function_call_dict)
    return function_calls

## Example: Parallel Function Calls on the Same Function (Asynchronous)

This example demonstrates how to perform parallel function calls asynchronously when you have a function that accepts one parameter per API call and you need to make repeated calls to that function.

With Parallel Function Calling and asynchronous operations, you can send a single API request to Gemini, receive multiple function call responses, make asynchronous external API calls, and return all responses to Gemini in bulk without extra configuration.

### Write Function Declarations and Wrap Them in a Tool

In [7]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
    ],
)

### Initialize the Gemini Model

In [8]:
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send Prompt to Gemini Asynchronously

In [9]:
prompt = "Search for articles related to solar panels, renewable energy, and battery storage and provide a summary of your findings"

In [10]:
import asyncio

async def get_response_async(chat, prompt):
    response = await chat.send_message_async(prompt)
    return response

response = asyncio.run(get_response_async(chat, prompt))

### Extract Function Names and Parameters

In [11]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'Solar panels'}},
 {'search_wikipedia': {'query': 'Renewable energy'}},
 {'search_wikipedia': {'query': 'Battery storage'}}]

### Make Asynchronous External API Calls

In [12]:
import asyncio

async def fetch_summary(query: str) -> str:
    loop = asyncio.get_event_loop()
    return await loop.run_in_executor(None, wikipedia.summary, query)

async def process_function_call(function_call: Dict[str, Dict[str, Any]]) -> str:
    print(function_call)
    function_name, function_args = next(iter(function_call.items()))
    if function_name == "search_wikipedia":
        query = function_args["query"]
        summary = await fetch_summary(query)
        return summary
    return ""

async def make_api_calls(function_calls: list) -> list:
    tasks = [process_function_call(fc) for fc in function_calls]
    return await asyncio.gather(*tasks)

api_responses = asyncio.run(make_api_calls(function_calls))

{'search_wikipedia': {'query': 'Solar panels'}}
{'search_wikipedia': {'query': 'Renewable energy'}}
{'search_wikipedia': {'query': 'Battery storage'}}


### Return API Responses to Gemini Asynchronously

In [13]:
async def return_to_gemini(chat, api_responses: list):
    parts = [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_responses[0],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_responses[1],
            },
        ),
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_responses[2],
            },
        ),
    ]
    response = await chat.send_message_async(parts)
    display(Markdown(response.text))

asyncio.run(return_to_gemini(chat, api_responses))

Solar panels are devices that convert sunlight into electricity using photovoltaic cells. They are arranged in arrays or systems and connected to inverters to convert DC electricity to AC electricity. Solar panels are a renewable and clean energy source that can reduce greenhouse gas emissions and lower electricity bills. However, they depend on sunlight availability, require cleaning, and have high initial costs.

Renewable energy refers to energy derived from renewable resources like sunlight, wind, and water. These sources are naturally replenished and have minimal environmental impact. Renewable energy systems have become increasingly efficient and affordable, leading to significant growth in their adoption. They offer a cleaner and more sustainable alternative to fossil fuels, mitigating climate change and reducing air pollution.

Battery storage power stations utilize batteries to store electrical energy, providing a rapid response to grid fluctuations and enhancing grid stability. They are essential for integrating renewable energy sources like solar and wind power, which have intermittent generation patterns. Battery storage systems can store excess energy generated during peak periods and release it when demand exceeds supply, ensuring a reliable and continuous power supply.

In summary, solar panels, renewable energy, and battery storage are interconnected aspects of a sustainable energy future. Solar panels harness solar energy, a renewable resource, while battery storage addresses the intermittency challenges associated with renewable energy sources. The combination of these technologies paves the way for a cleaner, more resilient, and sustainable energy system.

And you're done with no extra configuration necessary!

Note that Gemini will use the information in your `FunctionDeclarations` to determine if and when it should return parallel Function Call responses, or if it needs to make sequential calls based on dependencies. Ensure your logic accounts for this behavior.

## Example: Parallel Function Calls Across Multiple Functions (Asynchronous)

This example demonstrates how to perform parallel function calls asynchronously when you have multiple, independent functions. This reduces the number of consecutive Gemini API calls and can decrease the overall response time for the end user.

In this example, you'll use Parallel Function Calling in Gemini to retrieve multiple aspects of topics and articles on [Wikipedia](https://www.wikipedia.org/).

### Write Function Declarations and Wrap Them in a Tool

In [14]:
search_wikipedia = FunctionDeclaration(
    name="search_wikipedia",
    description="Search for articles on Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for on Wikipedia",
            },
        },
    },
)

suggest_wikipedia = FunctionDeclaration(
    name="suggest_wikipedia",
    description="Get suggested titles from Wikipedia for a given term",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Query to search for suggested titles on Wikipedia",
            },
        },
    },
)

summarize_wikipedia = FunctionDeclaration(
    name="summarize_wikipedia",
    description="Get article summaries from Wikipedia",
    parameters={
        "type": "object",
        "properties": {
            "topic": {
                "type": "string",
                "description": "Topic to summarize from Wikipedia",
            },
        },
    },
)

wikipedia_tool = Tool(
    function_declarations=[
        search_wikipedia,
        suggest_wikipedia,
        summarize_wikipedia,
    ],
)

### Initialize the Gemini Model

In [15]:
model = GenerativeModel(
    "gemini-1.5-pro",
    generation_config=GenerationConfig(temperature=0),
    tools=[wikipedia_tool],
)
chat = model.start_chat()

### Send Prompt to Gemini Asynchronously

In [16]:
prompt = "Show the search results, variations, and article summaries about Wikipedia articles related to the solar system"

In [17]:
response = asyncio.run(get_response_async(chat, prompt))

### Extract Function Names and Parameters

In [18]:
function_calls = extract_function_calls(response)
function_calls

[{'search_wikipedia': {'query': 'Solar System'}},
 {'suggest_wikipedia': {'query': 'Solar System'}},
 {'summarize_wikipedia': {'topic': 'Solar System'}}]

### Make Asynchronous External API Calls

In [19]:
async def process_function_call_multiple(function_call: Dict[str, Dict[str, Any]]) -> Any:
    print(function_call)
    function_name, function_args = next(iter(function_call.items()))
    if function_name == "search_wikipedia":
        query = function_args["query"]
        results = wikipedia.search(query)
        return ", ".join(results)
    elif function_name == "suggest_wikipedia":
        query = function_args["query"]
        suggestion = wikipedia.suggest(query)
        return suggestion or "No suggestions found"
    elif function_name == "summarize_wikipedia":
        topic = function_args["topic"]
        summary = await fetch_summary(topic)
        return summary
    return ""

async def make_api_calls_multiple(function_calls: list) -> Dict[str, Any]:
    tasks = [process_function_call_multiple(fc) for fc in function_calls]
    results = await asyncio.gather(*tasks)
    return {
        function_name: result
        for (function_call, result) in zip(function_calls, results)
        for function_name in function_call.keys()
    }

api_responses = asyncio.run(make_api_calls_multiple(function_calls))

{'search_wikipedia': {'query': 'Solar System'}}
{'suggest_wikipedia': {'query': 'Solar System'}}
{'summarize_wikipedia': {'topic': 'Solar System'}}


### Return API Responses to Gemini Asynchronously

In [20]:
async def return_to_gemini_multiple(chat, api_responses: Dict[str, Any]):
    parts = [
        Part.from_function_response(
            name="search_wikipedia",
            response={
                "content": api_responses.get("search_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="suggest_wikipedia",
            response={
                "content": api_responses.get("suggest_wikipedia", ""),
            },
        ),
        Part.from_function_response(
            name="summarize_wikipedia",
            response={
                "content": api_responses.get("summarize_wikipedia", ""),
            },
        ),
    ]
    response = await chat.send_message_async(parts)
    display(Markdown(response.text))

asyncio.run(return_to_gemini_multiple(chat, api_responses))

Here's some information about the solar system:

Search results: "Solar System", "Formation and evolution of the Solar System", "List of Solar System objects by size", "Photovoltaic system", "Solar System (disambiguation)", "Solar System belts", "Small Solar System body", "List of Solar System objects", "Passive solar building design", "List of natural satellites"

Suggested searches: "soler system"

Summary: The Solar System formed about 4.6 billion years ago. It contains the Sun, eight planets (Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune), at least nine dwarf planets, and countless smaller objects like asteroids and comets.  The Sun, a G-type main-sequence star, contains over 99.86% of the Solar System's mass.  The outer boundary of the Solar System is theorized to be the Oort cloud, extending up to 2,000,000 AU from the Sun.

And you're done! You successfully made parallel function calls across multiple functions asynchronously. Feel free to adapt the code samples here for your own use cases and applications. Explore other functionalities in the Gemini API by trying different notebooks.

Happy asynchronous parallel function calling!